# CNNs for Face Identification 

We will perform face identification using a subset of the [Labeled Faces in the Wild (LFW) dataset](http://vis-www.cs.umass.edu/lfw/). Face identification is a multi-class calssification problem where we have training/testing images for a set of subjects (classes). In 2014, researchers were able to surpass human accuracy in facial identification tasks using CNN models (along with advanced deep learning techniques)! 

We will observe the performance of a traditional face identification method, [Eigenfaces](https://en.wikipedia.org/wiki/Eigenface). We will then build a CNN model in order to surpass the accuracy of the traditional Eigenfaces technique. In the process, we will learn about the great power of CNNs!

### Requirements

1. Python 3.6
2. `pip install numpy`
3. `pip install sklearn`
4. `pip install matplotlib`
5. `pip install tensorflow==2.0.0`

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, MaxPool2D,concatenate, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)

Let's start by downloading and setting up a subset of the aligned and segmented version of the LFW dataset. We will only get subjects with at least 20 corresponding images. We will also get the grayscale versions of the images. This may take a few minutes . . .

In [ ]:
data_x, data_y = fetch_lfw_people(min_faces_per_person=20, color=False, return_X_y=True)
# normalize pixel values to [0,1]
data_x = data_x / 255.

The downloaded `data_x` will be the flattened vector representation of the images. 

We will reshape the vector respresentations back into the original grayscale images and store the images back in `data_x`.

In [ ]:
num_subjects = np.unique(data_y).shape[0]
print("Number of subjects: {}".format(np.unique(data_y).shape[0]))
print("Number of images: {}\n".format(data_y.shape[0]))

# image_x will contain the original grayscale images 
data_x = np.copy(data_x.reshape((data_x.shape[0], 62, 47, 1)))

print("data_x shape: {}".format(data_x.shape))
print("data_y shape: {}".format(data_y.shape))

As we can see, the dataset is made up of `3023` images of `62` subjects (or classes). 

Each image is a `62x47x1`-dimensional matrix containing 8-bit (0-255) grayscale pixel values.

Let's visualize a few of the grayscale images.

In [ ]:
np.random.seed(42)
for i in range(5):
    rnd_idx = np.random.choice(data_y.shape[0])
    rnd_img = data_x[rnd_idx]
    plt.figure()
    plt.imshow(rnd_img[:, :, 0], cmap="gray")

Now, let's split the dataset into train and test sets so we can perform face identification!

We will perform a `90/10` stratified split. This means that `90%` of each subject's images are placed in the train set, while the remaining `10%` of each subject's images are placed into the test set. 

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.1, random_state=42)

print("train_x shape: {}".format(train_x.shape))
print("train_y shape: {}\n".format(train_y.shape))

print("test_x shape: {}".format(test_x.shape))
print("test_y shape: {}".format(test_y.shape))

### Eigenfaces Overview

Eigenfaces is a face identification technique proposed in 1987. It was a very popular algorithm through the 1990's and is still used today as a baseline algorithm when gauging the performance of proposed face identification techniques. Eigenfaces is used to extract features from facial images. Such facial features are more useful in identification tasks than the corresponding oringinal images. The facial features can then be used for identification by being classified by a multi-class classifier, such a logistic regression model. 

### Eigenfaces Details

Eigenfaces is a Principcal Componet Analysis (PCA)-based dimensionality-reduction technique. Eigenfaces deals with the flattened vector representations of the face images. 

We give the vector representation of the training images as input to the algorithm. The output of the algoirthm is a set of Eigenfaces (vectors) which we can use to project vector representations of images into a lower-dimensional space which (hopefully) encode important facial features. 

Since the lower-dimensional space captures important facial features, Eigenfaces is used as a feature extraction technique. Given a facial image, we can reshape the image into a vector. Using the Eigenfaces, we reduce the dimensionality of the original vector into a vector which is more useful in identifying its corresponging subject. Therefore, we will use the Eigenfaces to extract facial features. Then, we will use a simple logistic regression classifier to classify the resulting facial features.

Let's start by flattening the training facial images. Then we will continue by computing the Eigenfaces which capture `95%` of the variance of the original dataset.

In [ ]:
# flatten images 
train_vector_x = train_x.reshape((train_x.shape[0], train_x.shape[1]*train_x.shape[2])) 
test_vector_x = test_x.reshape((test_x.shape[0], test_x.shape[1]*test_x.shape[2])) 

# retrieve eigenfaces
pca = PCA(n_components=0.95, random_state=42)
pca.fit(train_vector_x)
eigenfaces = pca.components_

print("Training vectors shape: {}".format(train_vector_x.shape))
print("Eigenfaces shape: {}".format(eigenfaces.shape))

As we see, if we flatten the training images to gather their vector representations, we recieve `2720` vectors of size `2914`. We also see that we have found `173` Eigenfaces vectors of size `2914`.

If we reshape the set of Eigenfaces to the shape of the original images, they reveal intesting ghost faces (i.e. "Eigenfaces").

Let's view the first `10` Eigenfaces.

In [ ]:
for i in range(10):
    eigenface = eigenfaces[i].reshape((train_x.shape[1], train_x.shape[2]))
    plt.figure()
    plt.imshow(eigenface, cmap="gray")

Spooky!

Now, let's use the Eigenfaces to extract facial features from the training and test set vectors. Since we have `173` Eigenfaces, we will reduce the dimensionality of the `2914`-dimensional train/test vectors to `173`-dimensional train/test feature vectors.

In [ ]:
train_features_x = np.dot(train_vector_x, eigenfaces.T)
test_features_x = np.dot(test_vector_x, eigenfaces.T)

print("Training vectors shape: {}".format(train_vector_x.shape))
print("Training features shape: {}".format(train_features_x.shape))
print("Testing vectors shape: {}".format(test_vector_x.shape))
print("Testing features shape: {}".format(test_features_x.shape))

Great! Now that we have extracted facial features from the train and testing set, let's perform facial identification using a simple logistic regression model!

In [ ]:
# specify the input size of the feature vectors
features = Input((train_features_x.shape[1],))
# a single fully-connected layer to map the features to a logit vector with one logit per subject
x = Dense(num_subjects)(features)
# use softmax activation to convert the logits to class probabilities for each subject
predictions = Activation("softmax")(x)

# create the model using the layers we defined previously
logistic_regression = Model(inputs=features, outputs=predictions)

# compile the model so that it uses Adam for optimization during training with cross-entropy loss
logistic_regression.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["acc"])

# print out a summary of the model achitecture
print(logistic_regression.summary())

In [ ]:
# since the subjects have different numbers of images, 
# we need to balance how much each subject contributes to the cross-entropy loss
class_weights = compute_class_weight("balanced", np.unique(train_y), train_y)

# train model, verbose set to 0 to supress output, wait a moment for training to complete
logistic_regression.fit(train_features_x, train_y, validation_data=(test_features_x, test_y), 
                        class_weight=class_weights, 
                        epochs=75, batch_size=128, verbose=0)

# evaluate model, get train/test accuracy
train_eig_pred = np.argmax(logistic_regression.predict(train_features_x), axis=1)
test_eig_pred = np.argmax(logistic_regression.predict(test_features_x), axis=1)
print("\nTraining accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(train_y, train_eig_pred)))
print("Testing accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(test_y, test_eig_pred)))

### Sample CNN Model
The Eigenfaces and logistic regression model doesn't perform that well on the unseen test set. Let's see how a simple CNN model compares.

Rather than dealing with vectors, we will begin working with the grayscale images directly. 

Note: CNN learns convolution filters which extract the most useful features. Then, a small neural network at the end of the CNN architecture performs the multi-class classification. Therefore, the CNN will make things less complex for us as the CNN will learn to perform both feature extraction and classification! 

Let's get going!

In [ ]:
# specifying the input size of the images
images = Input((train_x.shape[1], train_x.shape[2], 1,))
# a convolution layer of 32 filters of size 9x9 to extract features (valid padding)
x = Conv2D(32, padding="valid", kernel_size=(9,9))(images)
# a maxpooling layer to down-sample features with pool size (2, 2)
x = MaxPool2D(pool_size=(2,2))(x)
# another convolution layer of 64 filters of size 11x11 to extract features (valid padding)
x = Conv2D(64, padding="valid", kernel_size=(11,11))(x)
# a maxpooling layer to down-sample features with pool size (2, 2)
x = MaxPool2D(pool_size=(2,2))(x)
# flatten extracted features to form feature vector
x = Flatten()(x)
# a drop out layer for regularization (25% probability)
x = Dropout(0.25)(x)
# first fully-connected layer to map the features to vectors of size 256
x = Dense(256)(x)
# anoter drop out layer for regularization (25% probability)
x = Dropout(0.25)(x)
# a second fully-connected layer to map the features to a logit vector with one logit per subject
x = Dense(62)(x)
# use softmax activation to convert the logits to class probabilities for each subject
predictions = Activation(activation="softmax")(x)

# create the model using the layers we defined previously
sample_cnn = Model(inputs=images, outputs=predictions)

# compile the model so that it uses Adam for optimization during training with cross-entropy loss
sample_cnn.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["acc"])

# print out a summary of the model achitecture
print(sample_cnn.summary())

In [ ]:
# since the subjects have different numbers of images, 
# we need to balance how much each subject contributes to the cross-entropy loss
class_weights = compute_class_weight("balanced", np.unique(train_y), train_y)

# train model
sample_cnn.fit(train_x, train_y, validation_data=(test_x, test_y), 
               class_weight=class_weights, 
               epochs=15, batch_size=64, verbose=1)

# evaluate model, get train/test accuracy
train_pred = np.argmax(sample_cnn.predict(train_x), axis=1)
test_pred = np.argmax(sample_cnn.predict(test_x), axis=1)
print("\nTraining accuracy using sample CNN model: {}".format(accuracy_score(train_y, train_pred)))
print("Testing accuracy using sample CNN model: {}\n".format(accuracy_score(test_y, test_pred)))

print("The sample CNN model beat the Eigenfaces and logistic regression model! :)")
print("Training accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(train_y, train_eig_pred)))
print("Testing accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(test_y, test_eig_pred)))

## More tasks

2. Build three CNN models which surpass the traditional Eigenface method test accuracy. In each model use different architectures and optimizers. Points will be not be awarded if models architectures are not sufficiently different 
3. Build a fourth CNN model which achieves >70% test accuracy 
4. Display a correctly identified image and a training image corresponding to the correct training subject 
5. Display a incorrectly identified image and a training image corresponding to the correct training subject 

#### MODEL 1

In [ ]:
# Whats new?
# change padding to same
# tanh as the activation for conv and dense layers.
# Smaller kernel sizes and no dropout layer. lr = 0..001
# Using average pool instead of maxpool

images = Input((train_x.shape[1], train_x.shape[2], 1,))
x = Conv2D(64, padding="same", kernel_size=(5,5), activation="tanh")(images)
x = AveragePooling2D(pool_size=(2,2))(x)
x = Conv2D(32, padding="same", kernel_size=(5,5), activation="tanh")(x)
x = AveragePooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dense(128, activation = "tanh")(x)
x = Dense(128, activation = "tanh")(x)
x = Dense(62)(x)
predictions = Activation(activation="softmax")(x)

sample_cnn = Model(inputs=images, outputs=predictions)

sample_cnn.compile(optimizer=Adam(learning_rate = 0.001), loss="sparse_categorical_crossentropy", metrics=["acc"])

print(sample_cnn.summary())

In [ ]:
class_weights = compute_class_weight("balanced", np.unique(train_y), train_y)

# train model
sample_cnn.fit(train_x, train_y, validation_data=(test_x, test_y), 
               class_weight=class_weights, 
               epochs=25, batch_size=64, verbose=1)

# evaluate model, get train/test accuracy
train_pred = np.argmax(sample_cnn.predict(train_x), axis=1)
test_pred = np.argmax(sample_cnn.predict(test_x), axis=1)
print("\nTraining accuracy using sample CNN model: {}".format(accuracy_score(train_y, train_pred)))
print("Testing accuracy using sample CNN model: {}\n".format(accuracy_score(test_y, test_pred)))

print("Training accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(train_y, train_eig_pred)))
print("Testing accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(test_y, test_eig_pred)))

#### MODEL 2

In [ ]:
# whats new?
# change padding to same
# add relu activations
# change in hyperparameters
# 4 convolutions without dense layers

images = Input((train_x.shape[1], train_x.shape[2], 1,))
x = Conv2D(64, padding="same", kernel_size=(10,10), activation="relu")(images)
x = MaxPool2D(pool_size=(4,4))(x)
x = Conv2D(32, padding="same", kernel_size=(10,10), activation="relu")(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Conv2D(32, padding="same", kernel_size=(5,5), activation="relu")(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Conv2D(32, padding="same", kernel_size=(5,5), activation="relu")(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dropout(0.25)(x)
x = Dense(62)(x)
predictions = Activation(activation="softmax")(x)

sample_cnn = Model(inputs=images, outputs=predictions)

sample_cnn.compile(optimizer=RMSprop(), loss="sparse_categorical_crossentropy", metrics=["acc"])

print(sample_cnn.summary())

In [ ]:
class_weights = compute_class_weight("balanced", np.unique(train_y), train_y)

# train model
sample_cnn.fit(train_x, train_y, validation_data=(test_x, test_y), 
               class_weight=class_weights, 
               epochs=45, batch_size=64, verbose=1)

# evaluate model, get train/test accuracy
train_pred = np.argmax(sample_cnn.predict(train_x), axis=1)
test_pred = np.argmax(sample_cnn.predict(test_x), axis=1)
print("\nTraining accuracy using sample CNN model: {}".format(accuracy_score(train_y, train_pred)))
print("Testing accuracy using sample CNN model: {}\n".format(accuracy_score(test_y, test_pred)))

print("Training accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(train_y, train_eig_pred)))
print("Testing accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(test_y, test_eig_pred)))

#### MODEL3

In [ ]:
# whats new?
# Adding dense layer before convolution
# change padding to same
# add relu activations
# change in hyperparameters

num_filters = 50
filter_sizes = [(5,5),(7,7),(10, 10)]

images = Input((train_x.shape[1], train_x.shape[2], 1,))
pooled_outputs = []
for i in range(len(filter_sizes)):
    conv = Conv2D(num_filters, kernel_size=filter_sizes[i], padding='same')(images)
    conv = MaxPool2D(pool_size=(2,2))(conv)
    pooled_outputs.append(conv)
x = concatenate(pooled_outputs)
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(62)(x)
predictions = Activation(activation="softmax")(x)

sample_cnn = Model(inputs=images, outputs=predictions)

sample_cnn.compile(optimizer=Adagrad(learning_rate = 0.01), loss="sparse_categorical_crossentropy", metrics=["acc"])

print(sample_cnn.summary())

In [ ]:

class_weights = compute_class_weight("balanced", np.unique(train_y), train_y)

sample_cnn.fit(train_x, train_y, validation_data=(test_x, test_y), 
               class_weight=class_weights, 
               epochs=50, batch_size=64, verbose=1)

train_pred = np.argmax(sample_cnn.predict(train_x), axis=1)
test_pred = np.argmax(sample_cnn.predict(test_x), axis=1)
print("\nTraining accuracy using sample CNN model: {}".format(accuracy_score(train_y, train_pred)))
print("Testing accuracy using sample CNN model: {}\n".format(accuracy_score(test_y, test_pred)))

print("Training accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(train_y, train_eig_pred)))
print("Testing accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(test_y, test_eig_pred)))

#### MODEL 4

In [ ]:

images = Input((train_x.shape[1], train_x.shape[2], 1,))
x = Conv2D(32, padding="same", kernel_size=(10,10))(images)
x = MaxPool2D(pool_size=(4,4))(x)
x = Conv2D(64, padding="same", kernel_size=(5,5))(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dropout(0.25)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(62)(x)
predictions = Activation(activation="softmax")(x)

# create the model using the layers we defined previously
sample_cnn = Model(inputs=images, outputs=predictions)

# compile the model so that it uses Adam for optimization during training with cross-entropy loss
sample_cnn.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["acc"])

# print out a summary of the model achitecture
print(sample_cnn.summary())

In [ ]:
class_weights = compute_class_weight("balanced", np.unique(train_y), train_y)

# train model
sample_cnn.fit(train_x, train_y, validation_data=(test_x, test_y), 
               class_weight=class_weights, 
               epochs=25, batch_size=64, verbose=1)

# evaluate model, get train/test accuracy
train_pred = np.argmax(sample_cnn.predict(train_x), axis=1)
test_pred = np.argmax(sample_cnn.predict(test_x), axis=1)
print("\nTraining accuracy using sample CNN model: {}".format(accuracy_score(train_y, train_pred)))
print("Testing accuracy using sample CNN model: {}\n".format(accuracy_score(test_y, test_pred)))

print("Training accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(train_y, train_eig_pred)))
print("Testing accuracy using Eigenfaces and logistic regression model: {}".format(accuracy_score(test_y, test_eig_pred)))

In [ ]:
Display a correctly identified image and a training image corresponding to the correct training subject (1 pts.)
Display a incorrectly identified image and a training image corresponding to the correct training subject (1 pts.)

rnd_img = data_x[rnd_idx]
plt.figure()
plt.imshow(rnd_img[:, :, 0], cmap="gray")

#### Correct prediction

In [ ]:
test_pred = np.argmax(sample_cnn.predict(test_x), axis=1)
print("Is prediction for 1st image same as its label? ",test_pred[0] == test_y[0])
print("Class of image is", test_y[0])
print("Correctly tagged image - ")
rnd_img = test_x[0]
plt.figure()
plt.imshow(rnd_img[:, :, 0], cmap="gray")
plt.show()

print("\n\nImage of same class from train set")
train_img = train_x[np.where(train_y==test_y[0])[0][0]]
plt.figure()
plt.imshow(train_img[:, :, 0], cmap="gray")
plt.show()

#### Incorrect prediction

In [ ]:
# we see the 2nd test image is correctly predicted
test_pred = np.argmax(sample_cnn.predict(test_x), axis=1)
print("Is prediction for 1st image same as its label? ",test_pred[1] == test_y[1])
print("Class of image is", test_y[1], "  but predicted class is ", test_pred[1])
print("Incorrectly tagged image - ")
rnd_img = test_x[1]
plt.figure()
plt.imshow(rnd_img[:, :, 0], cmap="gray")
plt.show()

print("\n\nImage of same class from train set")
train_img = train_x[np.where(train_y==test_y[1])[0][0]]
plt.figure()
plt.imshow(train_img[:, :, 0], cmap="gray")
plt.show()

In [ ]:
#Complete the sample CNN as specified by the comments (2 pts.)....Done
#Build three CNN models which surpass the traditional Eigenface method test accuracy. In each model use different architectures and optimizers. Points will be not be awarded if models architectures are not sufficiently different (3 pts.)
#model-1: Done
#model-2: Done
#model-3: Done
#Build a fourth CNN model which achieves >70% test accuracy (3 pts.): i got 72% test accuracy
#Display a correctly identified image and a training image corresponding to the correct training subject (1 pts.): Done
#Display a incorrectly identified image and a training image corresponding to the correct training subject (1 pts.): Done

#..1,2,3 are the models and all have accuracy > 60
# 4th model the testing accuracy is more than 70%